# Evaluating Numerical Expressions

## Introduction

In this project, we will create a function similar to the eval() built-in function in python, and our goal is to use the stack data structure to evaluate the numerical expressions. 

## Linked List and Stack implementation

### Linked List

LinkedNode class will be the base of the stack class implementation.

In [1]:
# Linked list node implementation
class Node:
    def __init__(self, data):
        self.data = data
        self.prev = None
        self.next = None
        
# Linked List implementation
class LinkedList:
    def __init__(self):
        self.head = None
        self.tail = None
        self.length = 0
        
    def append(self, data):
        new_node = Node(data)
        if self.length == 0:
            self.head = self.tail = new_node
        else:
            self.tail.next = new_node
            new_node.prev = self.tail
            self.tail = new_node
        self.length += 1
        
    def __iter__(self):
        self._iter_node = self.head
        return self
    
    def __next__(self):
        if self.__iter_node is None:
            raise StopIteration
        ret = self._iter_node.data
        self._iter_node = self._iter_node.next
        return ret
    
    def prepend(self, data):
        new_node = Node(data)
        if self.length == 0:
            self.head = self.tail = new_node
        else:
            self.head.prev = new_node
            new_node.tail = self.head
            self.head = new_node
        self.length += 1
        
    def __len__(self):
        return self.length
    
    def __str__(self):
        return str([value for value in self])

### Stack 

In [2]:
# Stack data structure based on LinkedList
class Stack(LinkedList):
    def push(self, data):
        self.append(data)
        
    def peek(self):
        return self.tail.data
        
    def pop(self):
        ret = self.tail.data
        if self.length == 1:
            self.tail = self.head = None
        else:
            self.tail = self.tail.prev
            self.tail.nex = None
        self.length -= 1
        return ret

## Infix and Postfix Notation

Infix position means the operators between the operands as we usually write 1 + 1. 

However, it is easier to evaluate an expression written in postfix notation for a computer. This notation means the operands appearance before the operator as in 1 1 +. 

Also, we can evaluate an expression in postfix notation using a stack by:
* Reading the expression from left to right.
* Pushing numbers to the top of the stack.
* If we find an operator, we pop the top two elements and push the result of the operation.

To simplify the function we'll create, we will assume there are spaces between the elements, called tokens, of the string expression we want to evaluate.

In [3]:
# Transform the postfix expression string into a list os elements
def tokenize(expression):
    return expression.split()

In [4]:
# Test tokenize
expression = "12 2 4 + / 21 *"
tokenize(expression)

['12', '2', '4', '+', '/', '21', '*']

## Processing an Operator

When we calculate the expression, we need to make sure that we operate the two numbers in the correct order since the element at the top of the stack is the second operand in the expression and the element that is second to the top is the first operand. 

In [5]:
def process_minus(stack):
    second_element = stack.pop()
    first_element = stack.pop()
    stack.push(first_element - second_element)
    
def process_plus(stack):
    second_element = stack.pop()
    first_element = stack.pop()
    stack.push(first_element + second_element)
    
def process_times(stack):
    second_element = stack.pop()
    first_element = stack.pop()
    stack.push(first_element * second_element)

def process_divide(stack):
    second_element = stack.pop()
    first_element = stack.pop()
    stack.push(first_element / second_element)
    
def process_pow(stack):
    second_element = stack.pop()
    first_element = stack.pop()
    stack.push(first_element ** second_element)

## Evaluating Postfix Expressions

Now we will implement an algorithm to evaluate an expression in postfix notation.

In [6]:
# Evaluate expression in postfix notation
def evaluate_postfix(expression):
    tokens = tokenize(expression)
    stack = Stack()
    
    for token in tokens:
        if token == '+':
            process_plus(stack)
        elif token == '-':
            process_minus(stack)
        elif token == '*':
            process_times(stack)
        elif token == '/':
            process_divide(stack)
        elif token == '**':
            process_pow(stack)
        else:
            stack.push(float(token))
    return stack.pop()

In [7]:
# Test evaluate_postfix() function
expressions = [
    "4 6 -",
    "4 1 2 9 3 / * + 5 - *",
    "1 2 + 3 -",
    "1 2 - 3 +",
    "10 3 5 * 16 4 - / +",
    "5 3 4 2 - ** *",
    "12 2 4 + / 21 *",
    "1 1 + 2 **",
    "1 1 2 ** +"
]

for expression in expressions:
    print(evaluate_postfix(expression))

-2.0
8.0
0.0
2.0
11.25
45.0
42.0
4.0
2.0


## Operator Precedence in Infix Notation

Just as before, we'll assume that the infix expression string contains spaces between any two tokens.

To convert from infix to postfix, we'll implement the Shunting-yard algorithm using a stack. In this algorithm, we'll compare the precedence of operators.

In [8]:
# Operators precedence
precedence = {
    '+': 1,
    '-': 1,
    '*': 2,
    '/': 2,
    '**':3,
}

# Test precedence
precedence['/'] < precedence['-']

False

## From Infix to Postfix

We'll implement a function that, given a string with an expression in infix notation, outputs a string with the expression in postfix notation.

In our function, we need to do different things with different tokens:
* Opening parenthesis: Push the token into the stack for later.
* Closing parenthesis: Pop the top element and append it to the postfix token list while don't pop the opening parenthesis. After, we pop the opening parentheses out of the stack.
* Operators '+', '-', '\*', '/', '\**': While the top of the stack is also an operator with a precedence greater than or equal to this operator, pop the top element, and append it to the postfix token list. Push the current operator to the top of the stack
* Numbers: Append it to the postfix list.

Lastly, the stack may not be empty. In this case we pop all remaining operators into the postfix list.

In [9]:
# Process opening parenthesis
def process_opening_parenthesis(stack):
    stack.push('(')

In [10]:
# Process closing parenthesis
def process_closing_parenthesis(stack, postfix):
    while stack.peek() != '(':
        postfix.append(stack.pop())
    stack.pop() # Pop '('

In [11]:
# Handle operators
def process_operator(stack, postfix, operator):
    while len(stack) > 0 and stack.peek() in precedence and precedence[stack.peek()] >= precedence[operator]:
        postfix.append(stack.pop())
    
    stack.push(operator)

In [12]:
# Handle numbers
def process_number(postfix, number):
    postfix.append(number)

## Implementing the Shunting-yard Algorithm

We will gather all the pieces to convert an expression from infix to postfix notation.

In [13]:
# Convert infix expression in postfix
def infix_to_postfix(expression):
    tokens = tokenize(expression)
    stack = Stack()
    postfix = []
    for token in tokens:
        if token == '(':
            process_opening_parenthesis(stack)
        elif token == ')':
            process_closing_parenthesis(stack, postfix)
        elif token in ['+', '*', '-', '/', '**']:
            process_operator(stack, postfix, token)
        else:
            process_number(postfix, token)
            
    while len(stack) != 0:
        postfix.append(stack.pop())
        
    return ' '.join(postfix)

## Conclusion: Evaluating Infix Expressions

To conclude, we'll combine the two functions: infix to postfix notation and evaluation of an expression in postfix notation.

In [14]:
# Combine functions infix to postfix and evaluation of postfix expression
def evaluate(expression):
    postfix_expression = infix_to_postfix(expression)
    return evaluate_postfix(postfix_expression)

In [15]:
# Test evaluate function
expressions = [
    "1 + 1",
    "1 * ( 2 - ( 1 + 1 ) )",
    "4 * ( 1 + 2 * ( 9 / 3 ) - 5 )",
    "10 + 3 * 5 / ( 16 - 4 * 1 )",
    "2 * 2 * 2 * 2 * 2 * 2 * 2 * 2",
    "2 ** 2 ** 2 ** 2 ** 2",
    "( 1 - 2 ) / ( 3 - 5 )",
    "9 / 8 * 8",
    "64 / ( 8 * 8 )",
]

for expression in expressions:
    print(evaluate(expression))

2.0
0.0
8.0
11.25
256.0
65536.0
0.5
9.0
1.0
